# Rough Models Demo (rBergomi & Rough Heston)
* Self-contained notebook using functions from `src/rough.py`
* What you’ll see:
  * Path simulation sanity checks
  * Single-strike pricing + put–call parity
  * Smile from one set of terminal states
  * ATM IV term structure
* Toggle model with a single flag: `"rough_heston"` or `"rbergomi"`


In [ ]:
# 0) Imports & setup
import os, sys, math
import numpy as np
import matplotlib.pyplot as plt

# plotting style
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 12
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
from src.rough import (
    rbergomi_paths,
    rbergomi_euro_mc,
    rough_heston_paths,
    rough_heston_euro_mc,
)
from src.black_scholes import black_scholes_price

def implied_vol_from_price_bs(S, K, T, r, price, cp="call", q=0.0, lo=1e-4, hi=4.0, tol=1e-6, iters=80):
    lo = float(lo); hi = float(hi)
    for _ in range(iters):
        mid = 0.5*(lo+hi)
        p = black_scholes_price(S, K, T, r, mid, option_type=cp, q=q)
        if p > price: hi = mid
        else: lo = mid
        if hi - lo < tol: return mid
    return mid

# helper: price from a vector of terminal prices
def mc_price_from_ST(ST, K, r, T, cp="call"):
    if cp == "call":
        payoff = np.maximum(ST - K, 0.0)
    else:
        payoff = np.maximum(K - ST, 0.0)
    DF = math.exp(-r*T)
    disc = DF * payoff
    p = float(np.mean(disc))
    se = float(np.std(disc, ddof=1) / math.sqrt(ST.size))
    return p, se


## 1) Model parameters
Choose a baseline parameter set typical for equity under rough Heston.


In [7]:
S0    = 100.0
v0    = 0.04
r     = 0.02
q     = 0.00
H     = 0.10
kappa = 1.8
theta = 0.04
eta   = 1.8
rho   = -0.7

T     = 0.75
N     = 192
n_mc  = 12000
SEED  = 123


## 2) Simulate paths and sanity checks
Generate S(t) and V(t) paths, then inspect shapes, ranges, and plot a small subset.


In [8]:
t, S, V = rough_heston_paths(
    S0=S0, v0=v0, T=T, N=N, n_paths=n_mc, H=H,
    kappa=kappa, theta=theta, eta=eta, rho=rho,
    r=r, q=q, seed=SEED, batch_size=1024
)

print("Shapes:", S.shape, V.shape)
print("S min/max:", float(S.min()), float(S.max()))
print("V min/max:", float(V.min()), float(V.max()))

n_show = 30
idx = np.random.default_rng(7).choice(S.shape[0], size=n_show, replace=False)

fig, ax = plt.subplots()
ax.plot(t, S[idx].T, lw=0.7, alpha=0.6)
ax.set_title("Rough Heston spot paths")
ax.set_xlabel("t [years]")
ax.set_ylabel("S(t)")
plt.show()

fig, ax = plt.subplots()
ax.plot(t, V[idx].T, lw=0.7, alpha=0.6)
ax.set_title("Rough Heston variance paths")
ax.set_xlabel("t [years]")
ax.set_ylabel("V(t)")
plt.show()


TypeError: rbergomi_paths() got an unexpected keyword argument 'v0'. Did you mean 'S0'?